In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras import models
import matplotlib.pyplot as plt
import numpy as np
import pickle
from keras.models import model_from_json

In [23]:
from hier_networks import build_hrcnn_model

In [3]:
with open("/scratch/yz3464/NLP/data/history_cnn_1.pkl", "rb") as input_file:
    history = pickle.load(input_file)

In [6]:
model = models.load_model('/scratch/yz3464/NLP/checkpoints/cnn1.h5')

In [30]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [31]:
opt = Namespace(batch_size=10, char_embedd_dim=30, char_nbfilters=20, checkpoint_path='../checkpoint', dev='../data/dev.tsv', dropout=0.5, embedding='glove', embedding_dict='../data/glove.6B.50d.txt', embedding_dim=50, filter1_len=5, filter2_len=3, fine_tune=True, init_bias=False, l2_value=None, learning_rate=0.001, lstm_units=100, mode='mot', nbfilters=100, num_epochs=2, oov='embedding', optimizer='rmsprop', prompt_id=1, rnn_type='LSTM', test='../data/dev.tsv', train='../data/train.tsv', train_flag=True, use_char=False, vocab_size=4000)

In [35]:
model1 = build_hrcnn_model(opt, 4000, 10, 71, 50, 50, 50, None, True, init_mean_value)

Build model - INFO - Model parameters: max_sentnum = 71, max_sentlen = 50, embedding dim = 50, nbfilters = 100, filter1_len = 5, drop rate = 0.5
Build model - INFO - Use mean-over-time pooling on sentence


hier_networks.py:103: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(100, 5, padding="valid")`
  zcnn = TimeDistributed(Convolution1D(opts.nbfilters, opts.filter1_len, border_mode='valid'), name='zcnn')(resh_W)


Build model - INFO - Use mean-over-time pooling on text
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input (InputLayer)      (None, 3550)              0         
_________________________________________________________________
x (Embedding)                (None, 3550, 50)          200000    
_________________________________________________________________
x_maskedout (ZeroMaskedEntri (None, 3550, 50)          0         
_________________________________________________________________
drop_x (Dropout)             (None, 3550, 50)          0         
_________________________________________________________________
resh_W (Reshape)             (None, 71, 50, 50)        0         
_________________________________________________________________
zcnn (TimeDistributed)       (None, 71, 46, 100)       25100     
_________________________________________________________________
avg_zcnn (TimeDistri

hier_networks.py:143: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", name="output")`
  y = Dense(output_dim=1, activation='sigmoid', name='output')(avg_hz_lstm)
hier_networks.py:148: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ou..., inputs=Tensor("wo...)`
  model = Model(input=word_input, output=y)


In [52]:
model1.load_weights('/scratch/yz3464/NLP/checkpoints/hrcnn_mot_w.h5',by_name=True)

In [7]:
 from data_prepare import prepare_sentence_data

In [8]:
datapaths = ['/scratch/yz3464/NLP/data/data.tsv','/scratch/yz3464/NLP/data/dev.tsv','/scratch/yz3464/NLP/data/dev.tsv']
embedding_path = '/scratch/yz3464/NLP/data/glove.6B.50d.txt'

In [9]:
(X_t, Y_t, mask_train), (X_dev, Y_dev, mask_dev), (X_test, Y_test, mask_test), vocab, vocab_size, embed_table, overal_maxlen, overal_maxnum, init_mean_value = prepare_sentence_data(datapaths, \
 embedding_path, 'glove', 50, 1, 4000, tokenize_text=True, \
 to_lower=True, sort_by_len=False, vocab_path=None, score_index=6)

Loading data... - INFO - Prompt id is 1
Loading data... - INFO - Creating vocabulary from: /scratch/yz3464/NLP/data/data.tsv
Loading data... - INFO -   724840 total words, 16271 unique words
Loading data... - INFO -   Vocab size: 4000
Loading data... - INFO - Reading dataset from: /scratch/yz3464/NLP/data/data.tsv
Loading data... - INFO - Reading dataset from: /scratch/yz3464/NLP/data/dev.tsv
Loading data... - INFO - Reading dataset from: /scratch/yz3464/NLP/data/dev.tsv
Loading data... - INFO - Training data max sentence num = 71, max sentence length = 50
Loading data... - INFO - Dev data max sentence num = -1, max sentence length = -1
Loading data... - INFO - Test data max sentence num = -1, max sentence length = -1
Loading data... - INFO - Overall max sentence num = 71, max sentence length = 50
Prepare data ... - INFO - Statistics:
Prepare data ... - INFO -   train X shape: (1783, 71, 50)
Prepare data ... - INFO -   dev X shape:   (0, 71, 50)
Prepare data ... - INFO -   test X shape

data_prepare.py:32: RuntimeWarning: Mean of empty slice.
  dev_mean = y_dev.mean(axis=0)
/share/apps/numpy/1.13.1/intel/lib/python2.7/site-packages/numpy-1.13.1-py2.7-linux-x86_64.egg/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
data_prepare.py:34: RuntimeWarning: Mean of empty slice.
  test_mean = y_test.mean(axis=0)


Prepare data ... - INFO - OOV number =343, OOV ratio = 0.085771


In [10]:
nn1 = int(np.ceil(len(X_t)*0.7))
nn2 = int(np.ceil(len(X_t)*0.9))
 
Y_train = Y_t[0:nn1]
Y_dev = Y_t[nn1:nn2]
Y_test = Y_t[nn2:]
 
X_train = X_t.reshape((X_t.shape[0], X_t.shape[1]*X_t.shape[2]))[0:nn1]
X_dev = X_t.reshape((X_t.shape[0], X_t.shape[1]*X_t.shape[2]))[nn1:nn2]
X_test = X_t.reshape((X_t.shape[0], X_t.shape[1]*X_t.shape[2]))[nn2:]

In [11]:
pred = model.predict(X_test)

In [14]:
pred[175], Y_test[175]

(array([ 0.59979403], dtype=float32), array([ 0.60000002], dtype=float32))

In [12]:
np.abs(pred - Y_test).argmin()

175

In [15]:
X_test.shape

(178, 3550)

In [16]:
X_test[176]

array([108, 809,   4, ...,   0,   0,   0], dtype=int32)

In [17]:
text_list = []
with open('/scratch/yz3464/NLP/data/data.tsv',mode='r') as tst:
    tst.next()
    for line in tst:
        tokens = line.strip().split('\t')
        if int(tokens[1]) == 1:
            text_list.append(line)
    

In [18]:
len(text_list)

1783

In [19]:
test_text = text_list[nn2:]

In [20]:
len(X_t)

1783

In [21]:
test_text[175]

"1785\t1\tMy opinion is that people should have computers in their homes. Computers should be for important things like searching for jobs and other things. People can do their work in the computer. To teach your childrens how to use a computer or let their children go on the internet to they can search kids stuff. People shouldn't all their time on the computers and spend less time with their families and friends. I think people should spend more time with their families and their friends then spending it on the computer. But a lot of people use computers almost everyday. But a lot of people use computers almost everyday. Probably your family are worried about people spending their time everyday in the computer. They their friends how to type in the computer and teach them some stuff about the history. Do you think that people should alot more time with their families and be happy about it the they are spending time with their families. I think people shouldn't use the computer when t

In [48]:
pred_mot = model1.predict(X_test)

In [53]:
np.abs(pred_mot - Y_test)[0:175].argmin()

40

In [55]:
test_text[40]

'1650\t1\t"Dear local newspaper, @CAPS1 would a person do with out a computer? Well I think nothing because @CAPS1 if you are like me that have their family far away in another country well whith a computer you can see them throught a camara and talk to them just like if you where there. Computers just don\'t take your time they help you understand the menning of a family.These are many reasons why I think computers should be very important. My first reason is that they give you information about a place that you might be interested to see or know their culture. There are many people that like to explore and see new places, but they can\'t because maybe they just don\'t have time or they might have to much work. There is no exception for someone to not put their oportunities out, just because they don\'t have the time or a computer. My second reason is work. People like you that work in a office need information about a place that a disaster happened. You might need somenthing to see h

In [56]:
pred_mot[40], Y_test[40] #(array([ 0.72410029], dtype=float32), array([ 1.], dtype=float32))

(array([ 0.59877658], dtype=float32), array([ 0.60000002], dtype=float32))